In [7]:
'''Plan:
1. Load libraries, load data.
2. Preliminary EDA, dealing with missing values, merging train and test.
3. EDA, deleting variables.
4. Feature engineering, ohc.
5. Sample formation.
6. Feature scaling.
7. Model fitting.
8. Performance evaluation.
9. Predictions.
'''

# aside:
# when coding for interview ML purposes or Kaggle, never drop any obervations!
# you will have to make predictions for all obs in test sample.

# correct way to deal with missing obs and merge train and test samples:
# 1. Load both samples.
# 2. Impute missing values in both samples, using train sample to impute missing values.
# 3. Concatentate them into df.


### 1. Load libraries ###

import numpy as np
import pandas as pd
import os, warnings, random, time
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn import svm
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.impute import SimpleImputer

from xgboost import XGBClassifier

warnings.filterwarnings("ignore")
#os.getcwd()

In [8]:
### Load data ###

train = pd.read_csv('../input/tabular-playground-series-sep-2021/train.csv')
print(train.shape)

test = pd.read_csv('../input/tabular-playground-series-sep-2021/test.csv')

train = train.sample(n=50000)

(957919, 120)


In [9]:
### 2. ###

train.shape
train.describe()

# are there numerical features?

un_colval = pd.DataFrame([[x,len(train[x].unique())] for x in train.columns], columns = ['colname', 'n_unique'])
un_colval.loc[un_colval.n_unique < 100]
# all columns contain numerical features

,colname,n_unique
119,claim,2


In [ ]:
# given huge difference in accuracy with and without missing values, i suggest creating dummies for them.

#for i in df.columns:
#    df['mv'+i] = df[i].isnull()

In [10]:
#train.describe()
#test.describe()

# imputing missing values #

colmnames = train.columns
test['claim'] = np.nan
imp = SimpleImputer(missing_values=np.nan, strategy='median')
imp.fit(train)
train = pd.DataFrame(imp.transform(train))
test = pd.DataFrame(imp.transform(test))
#train.describe()

In [11]:
train['sample'] = 'train'
test['sample'] = 'pred'
df = pd.concat([train, test])
#df.colnames = list(colmnames)+'sample'
df

,0,1,2,3,4,5,6,7,8,9,...,111,112,113,114,115,116,117,118,119,sample
0,526662.0,0.100030,0.379480,4641.90,0.196980,0.187850,2.405300,2177.50,547840.0,2.779400e+14,...,1.7503,37.78100,6.60490,19338.00,1.1676,6.625100e+16,6816.30,0.40108,0.0,train
1,916455.0,0.089585,-0.013797,2675.90,0.182870,0.509330,1.768900,2106.20,218000.0,9.470900e+15,...,1.2556,-0.62244,7.58350,12690.00,1.2952,-1.086200e+14,7623.90,0.89290,0.0,train
2,658605.0,0.083463,0.119050,1235.70,0.003946,0.266620,0.569570,1304.40,175710.0,2.677800e+13,...,4.0337,-5.52050,-2.56720,-353.81,1.1646,-1.772300e+15,748.21,0.69200,0.0,train
3,260818.0,0.102180,-0.001167,538.54,0.167090,0.002191,0.817240,677.15,762580.0,3.677600e+15,...,2.8960,8.13580,-0.59556,94900.00,1.0729,3.363600e+15,347.08,0.31715,1.0,train
4,16585.0,0.103740,0.254490,202.39,0.137390,0.313980,-0.811050,1151.20,271000.0,-3.164700e+13,...,3.8834,21.48400,-2.59450,35700.00,1.4914,5.217900e+15,4696.00,0.24021,1.0,train
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
493469,1451388.0,-0.009112,0.308190,637.64,0.778200,0.414150,-1.068500,651.22,985000.0,6.079700e+15,...,2.3325,0.11226,-5.92380,19338.00,1.1559,8.163900e+16,857.09,1.56330,0.0,pred
493470,1451389.0,0.088922,0.482650,6924.10,0.025963,0.355400,-0.870200,2514.20,18004.0,6.073500e+14,...,1.7005,97.81300,4.37930,-2432.00,1.0707,4.691800e+16,7497.10,0.67075,0.0,pred
493471,1451390.0,0.140620,0.484750,1797.10,0.147020,0.288030,-1.407100,434.03,333050.0,2.351000e+15,...,1.6827,1.71750,8.06330,2471.40,1.1725,7.900900e+16,2904.60,0.18005,0.0,pred
493472,1451391.0,0.168000,0.351760,454.79,0.164580,0.169830,0.323850,2331.20,223980.0,-2.795300e+12,...,1.3531,155.21000,13.96300,-11.44,1.1946,-1.770600e+14,6763.10,0.33223,0.0,pred


In [12]:
newcolnames = list(colmnames) + ['sample']
df.columns = newcolnames
df

,id,f1,f2,f3,f4,f5,f6,f7,f8,f9,...,f111,f112,f113,f114,f115,f116,f117,f118,claim,sample
0,526662.0,0.100030,0.379480,4641.90,0.196980,0.187850,2.405300,2177.50,547840.0,2.779400e+14,...,1.7503,37.78100,6.60490,19338.00,1.1676,6.625100e+16,6816.30,0.40108,0.0,train
1,916455.0,0.089585,-0.013797,2675.90,0.182870,0.509330,1.768900,2106.20,218000.0,9.470900e+15,...,1.2556,-0.62244,7.58350,12690.00,1.2952,-1.086200e+14,7623.90,0.89290,0.0,train
2,658605.0,0.083463,0.119050,1235.70,0.003946,0.266620,0.569570,1304.40,175710.0,2.677800e+13,...,4.0337,-5.52050,-2.56720,-353.81,1.1646,-1.772300e+15,748.21,0.69200,0.0,train
3,260818.0,0.102180,-0.001167,538.54,0.167090,0.002191,0.817240,677.15,762580.0,3.677600e+15,...,2.8960,8.13580,-0.59556,94900.00,1.0729,3.363600e+15,347.08,0.31715,1.0,train
4,16585.0,0.103740,0.254490,202.39,0.137390,0.313980,-0.811050,1151.20,271000.0,-3.164700e+13,...,3.8834,21.48400,-2.59450,35700.00,1.4914,5.217900e+15,4696.00,0.24021,1.0,train
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
493469,1451388.0,-0.009112,0.308190,637.64,0.778200,0.414150,-1.068500,651.22,985000.0,6.079700e+15,...,2.3325,0.11226,-5.92380,19338.00,1.1559,8.163900e+16,857.09,1.56330,0.0,pred
493470,1451389.0,0.088922,0.482650,6924.10,0.025963,0.355400,-0.870200,2514.20,18004.0,6.073500e+14,...,1.7005,97.81300,4.37930,-2432.00,1.0707,4.691800e+16,7497.10,0.67075,0.0,pred
493471,1451390.0,0.140620,0.484750,1797.10,0.147020,0.288030,-1.407100,434.03,333050.0,2.351000e+15,...,1.6827,1.71750,8.06330,2471.40,1.1725,7.900900e+16,2904.60,0.18005,0.0,pred
493472,1451391.0,0.168000,0.351760,454.79,0.164580,0.169830,0.323850,2331.20,223980.0,-2.795300e+12,...,1.3531,155.21000,13.96300,-11.44,1.1946,-1.770600e+14,6763.10,0.33223,0.0,pred


In [13]:
### 3. ###

# we have many variables, are all of them useful? lets see correlation

#cors = df.corr()
#cors.loc[((cors > 0.8)&(cors<1)).any(1)]
# there are no pairwise correlations above 80%

In [14]:
### 4. ###

# check skew and possibly transform some variables #


In [15]:
### 5. ###

y_train = df.loc[df['sample']=='train',['claim']]
X_train = df.drop(columns=['claim'])
X_train = X_train.loc[X_train['sample']=='train']
X_pred = X_train.loc[X_train['sample']=='test']

print(y_train.shape, X_train.shape)

X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.1, random_state=2)

print(y_train.shape, X_train.shape, X_test.shape)

(50000, 1) (50000, 120)
(45000, 1) (45000, 120) (5000, 120)


In [16]:
X_train.drop(columns = ['sample'], inplace=True)
X_test.drop(columns = ['sample'], inplace=True)
X_pred.drop(columns = ['sample'], inplace=True)

In [17]:
X_train.shape

(45000, 119)

In [18]:
### 6. Scaling ###

ss = StandardScaler()

for i in X_train.columns:
    X_train[[i]] = ss.fit_transform(X_train[[i]])
    X_test[[i]] = ss.transform(X_test[[i]])

X_train.describe()  

,id,f1,f2,f3,f4,f5,f6,f7,f8,f9,...,f109,f110,f111,f112,f113,f114,f115,f116,f117,f118
count,4.500000e+04,4.500000e+04,4.500000e+04,4.500000e+04,4.500000e+04,4.500000e+04,4.500000e+04,4.500000e+04,4.500000e+04,4.500000e+04,...,4.500000e+04,4.500000e+04,4.500000e+04,4.500000e+04,4.500000e+04,4.500000e+04,4.500000e+04,4.500000e+04,4.500000e+04,4.500000e+04
mean,-3.607176e-17,2.762408e-16,-2.141694e-16,4.891396e-17,-1.479014e-16,-4.216726e-16,3.672618e-17,-8.881784e-17,1.320512e-16,-4.470251e-17,...,1.927347e-16,3.896019e-17,-1.901343e-16,-9.884131e-17,-3.305997e-17,1.244560e-17,3.673111e-17,6.038626e-17,3.010345e-16,-1.725521e-16
std,1.000011e+00,1.000011e+00,1.000011e+00,1.000011e+00,1.000011e+00,1.000011e+00,1.000011e+00,1.000011e+00,1.000011e+00,1.000011e+00,...,1.000011e+00,1.000011e+00,1.000011e+00,1.000011e+00,1.000011e+00,1.000011e+00,1.000011e+00,1.000011e+00,1.000011e+00,1.000011e+00
min,-1.738263e+00,-5.380555e+00,-2.494517e+00,-1.882106e+00,-1.293609e+00,-2.154454e+00,-5.917176e+00,-1.439784e+00,-1.170910e+00,-1.159069e+00,...,-1.450140e+00,-4.493991e+00,-1.862986e+00,-1.006227e+00,-2.802943e+00,-1.242087e+00,-2.641919e+00,-7.385613e-01,-1.391548e+00,-1.647440e+00
25%,-8.687421e-01,-4.550453e-01,-4.192776e-01,-5.624548e-01,-7.785014e-01,-4.345348e-01,-5.006805e-01,-8.902029e-01,-8.220552e-01,-7.610460e-01,...,-9.508583e-01,-4.628451e-01,-6.445105e-01,-5.303658e-01,-6.177043e-01,-6.485321e-01,-5.306581e-01,-6.328487e-01,-8.368305e-01,-6.822474e-01
50%,1.373289e-02,-9.853223e-04,2.926735e-01,-4.297261e-01,-2.978989e-01,1.567628e-01,-1.488625e-01,-1.323340e-01,-2.469617e-01,-5.529532e-01,...,-1.254630e-01,2.828197e-01,-4.545531e-01,-4.788344e-01,-7.860081e-02,-4.679427e-01,-2.680917e-01,-4.349680e-01,-2.257169e-01,-2.073797e-01
75%,8.627608e-01,5.900619e-01,7.633886e-01,3.170292e-02,4.442081e-01,7.379226e-01,4.663880e-01,6.832690e-01,5.178375e-01,5.422382e-01,...,7.506944e-01,7.794844e-01,4.337541e-01,-1.120554e-01,4.930463e-01,2.484507e-01,2.788489e-01,1.449907e-01,6.894510e-01,4.513891e-01
max,1.728839e+00,7.017956e+00,1.167265e+00,5.345938e+00,5.081339e+00,1.662834e+00,5.103952e+00,2.938078e+00,4.421749e+00,3.733693e+00,...,2.387433e+00,1.140958e+00,2.729458e+00,4.145431e+00,4.536720e+00,4.830804e+00,5.729936e+00,4.196304e+00,2.945402e+00,5.089434e+00


In [19]:
### 7. Model fitting ###

time1 = time.time()

log_rg = LogisticRegression()

grid_values = {'penalty': ['l2'], 'C': [0.00001, 0.0001, 0.001, 0.01]}

lr = GridSearchCV(log_rg, param_grid = grid_values, cv=4)
lr.fit(X_train, y_train)

print('logistic', lr.best_score_, lr.best_params_, time.time()-time1)
# after dropping missing I had 86%...

logistic 0.5225555555555556 {'C': 1e-05, 'penalty': 'l2'} 2.533942937850952


In [ ]:
time1 = time.time()

svm = svm.SVC(kernel='rbf')

grid_values = {'C':[0.01, 0.1, 1, 5, 10]}

svm = GridSearchCV(svm, param_grid = grid_values, cv=2)
svm.fit(X_train, y_train)

print('SVM', svm.best_score_, svm.best_params_, time.time()-time1)

In [31]:
time1 = time.time()

xgbcl = XGBClassifier(tree_method='gpu_hist', gpu_id=0 )

grid_values = {'n_estimators':[100,200,300],'eta':[0.15, 0.23, 0.3], 'max_depth':[2,3,4]}

xgb = GridSearchCV(xgbcl, param_grid = grid_values, cv=2)
xgb.fit(X_train, y_train)

print('XGBoost', xgb.best_score_, xgb.best_params_, time.time()-time1)

[20:32:29] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[20:32:29] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[20:32:30] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[20:32:30] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old be